In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import json

# ✅ Load mô hình
model = AutoModelForSequenceClassification.from_pretrained("/data/npl/ICEK/VACNIC/src/data/assest/phobert-finetune-hatespeech", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("/data/npl/ICEK/VACNIC/src/data/assest/phobert-finetune-hatespeech")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=1)

Device set to use cuda:0


In [3]:
import re
EXPLICIT_SLURS = {
    # ── Từ đơn ──
    "đm", "dm", "đmm", "đcm", "đcl",
    "địt", "đụ", "đĩ", "điếm", "lồn", "cặc",
    "chó", "khỉ", "lợn", "mọi",

    # ── Cụm ≥2 từ (giữ dấu cách) ──
    "địt mẹ", "đụ mẹ", "đụ má", "địt má", "đụ mẹ mày", "địt mẹ mày",
    "con chó", "lũ chó", "bọn chó", "chó chết", "chó má",
    "con khỉ", "lũ khỉ", "bọn khỉ",
    "con lợn", "lũ lợn", "bọn lợn",
    "con mọi", "thằng mọi", "bọn mọi", "lũ mọi", "mọi đen", "đám mọi", "đồ mọi", "mọi rợ",
    "con đĩ", "con điếm", "đồ đĩ", "đồ điếm", "đĩ rẻ", "đĩ chúa"
}

In [9]:
data_list = []
with open('/data/npl/ICEK/VACNIC/data/backup/LLMs/official_test.jsonl', 'r', encoding='utf-8') as infile:
    i = 0
    for line in infile:
        if i <3069:
            item = json.loads(line)
            # Chỉ lấy translation và label
            data_list.append({
                'text': item['translation'],
                'label': item['label']
            })
        i+=1
df_data = pd.DataFrame(data_list)

In [8]:
df_data[:5]

,text,label,prediction,true_label,final_prediction
0,Bắt chước phong cách âm nhạc và cử chỉ của họ ...,Implicit HS,LABEL_2,LABEL_2,Implicit HS
1,"David Allen Snyder, 26 tuổi, đã qua đời vào ng...",Non HS,LABEL_0,LABEL_0,Non HS
2,Người Syria thì không biết quý trọng mạng sống...,Explicit HS,LABEL_2,LABEL_2,Explicit HS
3,Dân tộc nào tồn tại hàng ngàn năm mà vẫn luôn ...,Non HS,LABEL_0,LABEL_0,Non HS
4,"Nếu bạn có vấn đề với cô ấy, chắc là vì bạn gh...",Implicit HS,LABEL_0,LABEL_2,Non HS


In [10]:
 def is_explicit(text: str) -> bool:
    """
    Trả về True nếu text chứa bất kỳ slur nằm trong EXPLICIT_SLURS.
    Không phân biệt hoa thường; tách từ bằng regex \w+.
    """
    tokens = re.findall(r"\w+", text.lower())
    return any(tok in EXPLICIT_SLURS for tok in tokens)

def zero_shot_prediction_df(df):
    preds = []
    for text in tqdm(df['text'], desc="Đang phân loại one-shot"):

        result = pipe(text)[0][0]['label']  
        preds.append(result)

    df['prediction'] = preds
    df['true_label'] = df['label'].map(
        lambda x: 'LABEL_2' if x in ('Implicit HS', 'Explicit HS') else 'LABEL_0'
    )

    def get_final_pred(row):
        label = row['label']      
        pred  = row['prediction']   
        text  = row['text']

        if label == 'Implicit HS' and pred == 'LABEL_2':
            return 'Implicit HS'
        elif label == 'Explicit HS' and pred == 'LABEL_2':
            return 'Explicit HS'

        # hate thật nhưng mô hình bảo 'clean'
        elif label in ('Implicit HS', 'Explicit HS') and pred == 'LABEL_0':
            return 'Non HS'

        # non thật + mô hình clean
        elif label == 'Non HS' and pred == 'LABEL_0':
            return 'Non HS'

        # ---- TH còn lại: label = non & pred = hate ----
        else: return 'Explicit HS' if is_explicit(text) else 'Implicit HS'

    df['final_prediction'] = df.apply(get_final_pred, axis=1)
    return df


df_zero = zero_shot_prediction_df(df_data)

Đang phân loại one-shot: 100%|██████████| 3063/3063 [00:32<00:00, 94.26it/s] 


In [11]:
print(classification_report(df_zero['label'],df_zero['final_prediction']))

              precision    recall  f1-score   support

 Explicit HS       0.99      0.95      0.97       982
 Implicit HS       0.61      0.47      0.53      1090
      Non HS       0.52      0.68      0.59       991

    accuracy                           0.69      3063
   macro avg       0.71      0.70      0.70      3063
weighted avg       0.70      0.69      0.69      3063

